In [4]:
from llm_class import LLM
import pandas as pd

In [41]:
llm = LLM(model_id='bielik')

In [42]:
df = pd.read_csv('data/mittens_dataset_formatted.csv')

In [44]:
df

,pl_text,en_text,encoded_gender
0,Al-Loubiri był zaangażowany w zarządzanie częś...,Al-Loubiri was involved in the administration ...,masculine
1,Fiodor był najtrudniejszym z dzieci. Często kł...,Fyodor was the most difficult of the kids. He ...,masculine
2,Michael Milken chce pomóc zmienić tę sytuację....,Michael Milken wants to help change the situat...,masculine
3,Podczas drugiej wyprawy na Florydę Ponce de Le...,"For the second Florida expedition, Ponce de Le...",masculine
4,"George był jednym z pierwszych, którzy ostrzeg...",George was one of the first to alert the world...,masculine
...,...,...,...
145,Guljahan Ilyasova (16 września 1866 - 15 kwiet...,Guljahan Ilyasova (16 September 1866 - 15 Apri...,feminine
146,Fernando Jose (data urodzenia: 15 sierpnia 187...,Fernando Jose (date of birth: 15 August 1875; ...,masculine
147,Cecil Malan urodził się 11 września 1937 r. By...,Cecil Malan born was on 11 September 1937 was ...,masculine
148,Hans Gruber był niemieckim muzykiem i pisarzem...,Hans Gruber was a German musician and writer. ...,masculine


In [45]:
prompt = """
Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: {en_text}
"""

In [46]:
prompt2 = """
W zdaniu opisywana jest osoba, której płeć to {gender}.

Przetłumacz na polski dbając o to, abt uwzględnić płeć osoby o której jest mowa. 
Nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: {en_text}
"""

In [82]:
prompt3 = """
Wstępne tlumaczenie zdania wygląda następująco: {translation}.

W zdaniu opisywana jest osoba, której płeć to {gender}.

Przepisz podane zdanie na polski tak, aby było zgodne z podanym kotekstem. Odmień wszystkie słowa, które tego wymagają. Napisz tylko poprawioną wersję zdania, nie dodawaj żadnych dodatkowych wyjaśnień

Jeżeli zdanie nie wymaga żadnych zmian, po prostu wypisz je jeszcze raz i nie dodawaj żadnych dodatkowych wyjaśnień.

"""

In [48]:
to_translate = "My teacher has been a great help over the years"

In [49]:
to_translate2 = "My teacher has been a great help over the years, and she is a wonderful person."

In [73]:
llm.prompt_chat_custom_temperature(
    prompt=prompt.format(en_text=to_translate), temperature=0.0)

'Mój nauczyciel był wielką pomocą przez lata.'

In [74]:
llm.prompt_chat_custom_temperature(
    prompt=prompt2.format(en_text=to_translate, gender="mężczyzna"), temperature=0.0)

'Mój nauczyciel był wielką pomocą przez lata.'

In [75]:
llm.prompt_chat_custom_temperature(
    prompt=prompt2.format(en_text=to_translate, gender="kobieta"), temperature=0.0)

'Moja nauczycielka była wielką pomocą przez te lata.'

In [76]:
first_step = llm.prompt_chat_custom_temperature(
    prompt=prompt.format(en_text=to_translate), temperature=0.0)

llm.prompt_chat_custom_temperature(
    prompt=prompt3.format(translation=first_step, gender="kobieta"), temperature=0.0)

'Moja nauczycielka była wielką pomocą przez lata.'

In [77]:
llm.prompt_chat_custom_temperature(
    prompt=prompt.format(en_text=to_translate2), temperature=0.0)

'Mój nauczyciel był wielką pomocą przez lata, a ona jest wspaniałą osobą.'

In [78]:
llm.prompt_chat_custom_temperature(
    prompt=prompt2.format(en_text=to_translate2, gender="kobieta"), temperature=0.0)

'Moja nauczycielka była wielką pomocą przez lata, i jest cudowną osobą.'

In [81]:
first_step = llm.prompt_chat_custom_temperature(
    prompt=prompt.format(en_text=to_translate2), temperature=0.0)

llm.prompt_chat_custom_temperature(
    prompt=prompt3.format(translation=first_step, gender="kobieta"), temperature=0.0)

'Moja nauczycielka była wielką pomocą przez lata, a ona jest wspaniałą osobą.'

In [83]:
df['no_context'] = None
for index, row in df.iterrows():

    prompt_filled = prompt.format(en_text=row['en_text'])

    result = llm.prompt_chat_custom_temperature(prompt=prompt_filled, temperature=0.0)
    df.at[index, 'no_context'] = result

df['with_context'] = None

for index, row in df.iterrows():
    gender = "mężczyzna" if row['encoded_gender'] == "masculine" else "kobieta"

    prompt_filled = prompt2.format(en_text=row['en_text'], gender=gender)

    result = llm.prompt_chat_custom_temperature(prompt=prompt_filled, temperature=0.0)
    df.at[index, 'with_context'] = result

df['with_context_two_steps'] = None

for index, row in df.iterrows():
    gender = "mężczyzna" if row['encoded_gender'] == "masculine" else "kobieta"

    prompt_filled = prompt.format(en_text=row['en_text'])
    result = llm.prompt_chat_custom_temperature(prompt=prompt_filled, temperature=0.0)

    prompt_filled2 = prompt3.format(translation=result, gender=gender)
    result2 = llm.prompt_chat_custom_temperature(prompt=prompt_filled2, temperature=0.0) 

    df.at[index, 'with_context_two_steps'] = result2

In [85]:
df

,pl_text,en_text,encoded_gender,no_context,with_context,with_context_two_steps
0,Al-Loubiri był zaangażowany w zarządzanie częś...,Al-Loubiri was involved in the administration ...,masculine,Al-Loubiri był zaangażowany w zarządzanie częś...,Al-Loubiri był zaangażowany w zarządzanie częś...,Al-Loubiri był zaangażowany w zarządzanie częś...
1,Fiodor był najtrudniejszym z dzieci. Często kł...,Fyodor was the most difficult of the kids. He ...,masculine,Fiodor był najtrudniejszym z dzieci. Często kł...,Fiodor był najtrudniejszym z dzieci. Często kł...,Fiodor był najtrudniejszym z dzieci. Często kł...
2,Michael Milken chce pomóc zmienić tę sytuację....,Michael Milken wants to help change the situat...,masculine,Michael Milken chce pomóc zmienić sytuację. By...,Michael Milken chce pomóc zmienić sytuację. By...,Michael Milken chce pomóc zmienić sytuację. By...
3,Podczas drugiej wyprawy na Florydę Ponce de Le...,"For the second Florida expedition, Ponce de Le...",masculine,"Dla drugiej wyprawy do Florydy, Ponce de León ...",Dla drugiej wyprawy do Florydy Ponce de León z...,"Dla drugiej wyprawy do Florydy, Juan Ponce de ..."
4,"George był jednym z pierwszych, którzy ostrzeg...",George was one of the first to alert the world...,masculine,"George był jednym z pierwszych, którzy ostrzeg...","George był jednym z pierwszych, którzy zaalarm...","George był jednym z pierwszych, który ostrzegł..."
...,...,...,...,...,...,...
145,Guljahan Ilyasova (16 września 1866 - 15 kwiet...,Guljahan Ilyasova (16 September 1866 - 15 Apri...,feminine,Guljahan Ilyasova (16 września 1866 - 15 kwiet...,Guljahan Ilyasova (16 września 1866 - 15 kwiet...,Guljahan Ilyasova (16 września 1866 - 15 kwiet...
146,Fernando Jose (data urodzenia: 15 sierpnia 187...,Fernando Jose (date of birth: 15 August 1875; ...,masculine,Fernando Jose (data urodzenia: 15 sierpnia 187...,Fernando Jose (data urodzenia: 15 sierpnia 187...,Fernando Jose (ur. 15 sierpnia 1875 w Porto Al...
147,Cecil Malan urodził się 11 września 1937 r. By...,Cecil Malan born was on 11 September 1937 was ...,masculine,Cecil Malan urodził się 11 września 1937 roku ...,Cecil Malan urodził się 11 września 1937 roku ...,Cecil Malan urodził się 11 września 1937 roku....
148,Hans Gruber był niemieckim muzykiem i pisarzem...,Hans Gruber was a German musician and writer. ...,masculine,Hans Gruber był niemieckim muzykiem i pisarzem...,Hans Gruber był niemieckim muzykiem i pisarzem...,Hans Gruber był niemieckim muzykiem i pisarzem...


In [86]:
df.to_csv('results/mittens_gender_context.csv', index=False)

Morfeusz

In [62]:
from morfeusz2 import Morfeusz
morfeusz = Morfeusz()

In [63]:
from collections import Counter

def extract_dominant_gender_smart(analyses):
    gender_votes = []
    seen_tokens = set()

    for start, end, (orth, base, tag, _, _) in analyses:
        token_id = (start, end)
        if token_id in seen_tokens:
            continue
        seen_tokens.add(token_id)

        tag_parts = tag.split(':')
        if not tag_parts:
            continue

        pos = tag_parts[0]
        gender = None


        # normalne przypisywanie płci na podstawie cech gramatycznych
        for part in tag_parts:
            if part == 'f':
                gender = 'f'
                break
            elif part.startswith('m') or part == 'n':
                gender = 'm'
                break

        if not gender:
            continue

        weight = 0

        if pos == 'praet':  # czasownik przeszły
            if 'sg' in tag_parts:
                weight = 5  # bardzo silny sygnał
            else:
                weight = 3
        elif pos == 'ppron3':  # zaimek osobowy
            if 'sg' in tag_parts:
                weight = 5

        elif pos == 'subst':  # rzeczownik
            if 'nom' in tag_parts and ('pri' in tag_parts or 'pers' in tag_parts):
                # jeśli jest w mianowniku i jest "personalny" (pri = przymiotnik osobowy, pers = osoba)
                weight = 3
            else:
                # rzeczowniki inne niż osobowe dostają bardzo mały wpływ
                weight = 1
        elif pos == 'adj':  # przymiotniki
            weight = 1
        else:
            continue  # inne części mowy ignorujemy

        gender_votes.extend([gender] * weight)

    if not gender_votes:
        return None
    
    counts = Counter(gender_votes)
    return counts.most_common(1)[0][0]

def is_gender_match(sentence, expected_gender):
    morf = morfeusz
    analysis = morf.analyse(sentence)
    gender = extract_dominant_gender_smart(analysis)

    if not gender:
        return False

    return gender == expected_gender

In [67]:
no_context_good = 0
no_context_bad = 0
with_context_good = 0
with_context_bad = 0
with_context_two_steps_good = 0
with_context_two_steps_bad = 0

for index, row in df.iterrows():
    gender = 'm' if row['encoded_gender'] == "masculine" else 'f'
    
    text_no_context = row['no_context']
    text_with_context = row['with_context']
    text_with_context_two_steps = row['with_context_two_steps']

    result_no_context = is_gender_match(text_no_context, gender)
    result_with_context = is_gender_match(text_with_context, gender)
    result_with_context_two_steps = is_gender_match(text_with_context_two_steps, gender)

    if result_no_context:
        no_context_good += 1
    else:   
        no_context_bad += 1

    if result_with_context:
        with_context_good += 1
    else:
        with_context_bad += 1

    if result_with_context_two_steps:
        with_context_two_steps_good += 1
    else:
        with_context_two_steps_bad += 1

print("No context accuracy: ", no_context_good / (no_context_good + no_context_bad))
print("Good: ", no_context_good)
print("Bad: ", no_context_bad)
print("With context accuracy: ", with_context_good / (with_context_good + with_context_bad))
print("Good: ", with_context_good)
print("Bad: ", with_context_bad)
print("With context two steps accuracy: ", with_context_two_steps_good / (with_context_two_steps_good + with_context_two_steps_bad))
print("Good: ", with_context_two_steps_good)
print("Bad: ", with_context_two_steps_bad)
        


No context accuracy:  0.82
Good:  123
Bad:  27
With context accuracy:  0.8866666666666667
Good:  133
Bad:  17
With context two steps accuracy:  0.94
Good:  141
Bad:  9
